In [ ]:
!pip3 install torch==1.7.0 -q
!pip3 install transformers==2.9.0 -q
!pip3 install pytorch_lightning==0.7.5 -q
!pip3 install gdown -q
!mkdir t5-p

import gdown 
import os.path
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

# model_location = "https://drive.google.com/LINK_TO_MODEL"
# config_location = "https://drive.google.com/LINK_TO_MODEL"
# if os.path.isfile(datamodel):
#     ('Model was already downloaded.')
# else:
#     gdown.download(model_location, datamodel, True)
#     gdown.download(config_location, config_file, True)

In [ ]:
datamodel = 't5-p/pytorch_model.bin'
config_file = 't5-p/config.json'

In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('./t5-p/')

In [ ]:
with open('custom_corpus/input.txt') as f:
    mylist = f.read().splitlines()

print('Number of sentences to be paraphrased: ', len(mylist))

In [ ]:
inputs = []
final_outputs = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
for sentence in mylist: 
    max_len = 256

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    beam_outputs = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=1
    )   

    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            inputs.append(sentence)
            final_outputs.append(sent)

In [ ]:
dict = {'Inputs': inputs, 'Paraphrase': final_outputs}

In [ ]:
!mkdir synthetic_data

In [ ]:
import pandas as pd
df = pd.DataFrame(dict)
df.to_csv('synthetic_data/synthetic_paraphrases.csv')